In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


In [2]:
TRAIN_PATH = "/kaggle/input/playground-series-s4e2/train.csv"
train = pd.read_csv(TRAIN_PATH)
train.head()
# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

TEST_PATH = "/kaggle/input/playground-series-s4e2/test.csv"
test = pd.read_csv(TEST_PATH)
test.head()

Number of data points in train: 20758
Number of features in train: 18


,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,20758,Male,26.899886,1.848294,120.644178,yes,yes,2.938616,3.000000,Sometimes,no,2.825629,no,0.855400,0.000000,Sometimes,Public_Transportation
1,20759,Female,21.000000,1.600000,66.000000,yes,yes,2.000000,1.000000,Sometimes,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation
2,20760,Female,26.000000,1.643355,111.600553,yes,yes,3.000000,3.000000,Sometimes,no,2.621877,no,0.000000,0.250502,Sometimes,Public_Transportation
3,20761,Male,20.979254,1.553127,103.669116,yes,yes,2.000000,2.977909,Sometimes,no,2.786417,no,0.094851,0.000000,Sometimes,Public_Transportation
4,20762,Female,26.000000,1.627396,104.835346,yes,yes,3.000000,3.000000,Sometimes,no,2.653531,no,0.000000,0.741069,Sometimes,Public_Transportation


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # for data visualization purposes
import seaborn as sns # for statistical data visualization
%matplotlib inline
import imblearn
import datetime

In [4]:
data_train = train.copy()
data_test = test.copy()

In [5]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13840 entries, 0 to 13839
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              13840 non-null  int64  
 1   Gender                          13840 non-null  object 
 2   Age                             13840 non-null  float64
 3   Height                          13840 non-null  float64
 4   Weight                          13840 non-null  float64
 5   family_history_with_overweight  13840 non-null  object 
 6   FAVC                            13840 non-null  object 
 7   FCVC                            13840 non-null  float64
 8   NCP                             13840 non-null  float64
 9   CAEC                            13840 non-null  object 
 10  SMOKE                           13840 non-null  object 
 11  CH2O                            13840 non-null  float64
 12  SCC                             

In [6]:
data_train.drop(['id'], axis=1, inplace=True)

data_train['Gender'] = data_train['Gender'].map({'Male': 1, 'Female': 0})
data_train['family_history_with_overweight'] = data_train['family_history_with_overweight'].map({'yes': 1, 'no': 0})
data_train['FAVC'] = data_train['FAVC'].map({'yes': 1, 'no': 0})
data_train['CAEC'] = data_train['CAEC'].map({'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3})
data_train['SMOKE'] = data_train['SMOKE'].map({'yes': 1, 'no': 0})
data_train['SCC'] = data_train['SCC'].map({'yes': 1, 'no': 0})
data_train['CALC'] = data_train['CALC'].map({'no': 0, 'Sometimes': 1, 'Frequently': 2})
data_train['MTRANS'] = data_train['MTRANS'].map({'Bike': 1, 'Motorbike': 2, 'Walking': 3, 'Automobile': 4, 'Public_Transportation': 5})

In [7]:
data_test.drop(['id'], axis=1, inplace=True)

data_test['Gender'] = data_test['Gender'].map({'Male': 1, 'Female': 0})
data_test['family_history_with_overweight'] = data_test['family_history_with_overweight'].map({'yes': 1, 'no': 0})
data_test['FAVC'] = data_test['FAVC'].map({'yes': 1, 'no': 0})
data_test['CAEC'] = data_test['CAEC'].map({'no': 0, 'Sometimes': 1, 'Frequently': 2, 'Always': 3})
data_test['SMOKE'] = data_test['SMOKE'].map({'yes': 1, 'no': 0})
data_test['SCC'] = data_test['SCC'].map({'yes': 1, 'no': 0})
data_test['CALC'] = data_test['CALC'].map({'no': 0, 'Sometimes': 1, 'Frequently': 2})
data_test['MTRANS'] = data_test['MTRANS'].map({'Bike': 1, 'Motorbike': 2, 'Walking': 3, 'Automobile': 4, 'Public_Transportation': 5})

In [8]:
data_test['CALC'] = data_test['CALC'].fillna(data_train['CALC'].mean())

In [9]:
from sklearn.preprocessing import LabelEncoder
le2=LabelEncoder()
X_train=data_train.drop('NObeyesdad',axis=1)
Y_train=le2.fit_transform(data_train['NObeyesdad'])

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X1 = scaler.fit_transform(X_train)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
X_train_lda = lda.fit_transform(X1, Y_train)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#classifier = LogisticRegression(penalty='l2', C=0.6, solver='liblinear', random_state=42)
classifier = LogisticRegression(multi_class='multinomial', random_state=42, max_iter=1000)

In [12]:
classifier.fit(X_train_lda, Y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=42)

In [13]:
data_test_scaled = scaler.transform(data_test)

In [14]:
X_test_lda = lda.transform(data_test_scaled)

In [15]:
classifier.fit(X_train_lda, Y_train)
Y_test_pred = classifier.predict(X_test_lda)

predictions=le2.inverse_transform(Y_test_pred)

In [16]:
submission = pd.read_csv("/kaggle/input/playground-series-s4e2/sample_submission.csv")
submission["NObeyesdad"] = predictions
submission.to_csv("submission1.csv", index=False)
submission.head()

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III
